# Sampling from Databases

This notebook will create samples from the two databases built from scraping.

In [1]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

**Note: This same dataset was used for another project that incorporated weather data, but this was not used in DMW.**

In [4]:
w_df = pd.read_csv('chicago-weather.csv', parse_dates=['StartTime(UTC)',
                                                       'EndTime(UTC)'])
w_df.head()

,StartTime(UTC),EndTime(UTC),Type,Severity
0,2016-01-01 11:00:00,2016-01-01 13:00:00,Snow,Light
1,2016-01-03 23:00:00,2016-01-04 06:00:00,Snow,Light
2,2016-01-04 15:30:00,2016-01-04 17:00:00,Snow,Light
3,2016-01-04 18:00:00,2016-01-05 02:00:00,Snow,Light
4,2016-01-08 08:00:00,2016-01-08 14:15:00,Rain,Light


## TNP Dataset

Since the TNP Dataset was scraped monthly into individual databases, we'll iterate over each database and randomly sample 2% from each month.

In [1]:
tnp_dir = '/mnt/processed/private/msds2021/lt6/chicago-dataset/tnp/'
datasets = ['tnp_201901.db',
            'tnp_201902.db',
            'tnp_201903.db',
            'tnp_201904.db',
            'tnp_201905.db',
            'tnp_201906.db',
            'tnp_201907.db',
            'tnp_201908.db',
            'tnp_201909.db',
            'tnp_201910.db',
            'tnp_201911.db',
            'tnp_201912.db']

In [3]:
cols = ['trip_id', 'trip_start_timestamp', 'trip_end_timestamp', 'trip_seconds',
       'trip_miles', 'pickup_community_area', 'dropoff_community_area', 'fare', 'tip',
       'additional_charges', 'trip_total', 'shared_trip_authorized',
       'trips_pooled', 'pickup_centroid_latitude', 'pickup_centroid_longitude',
       'dropoff_centroid_latitude', 'dropoff_centroid_longitude']

In [6]:
for d in datasets:
    with sqlite3.connect(tnp_dir + d) as conn:
        print('START:', d)
        sql = f"""SELECT * FROM trips WHERE trip_id IN
                    (SELECT trip_id FROM trips ORDER BY RANDOM()
                     LIMIT (SELECT ROUND(COUNT(*) * 0.02) FROM trips))"""
        df = pd.read_sql(sql, conn,
                         parse_dates=['trip_start_timestamp',
                                      'trip_end_timestamp']).loc[:, cols]

        # Build the Weather Mapper
        wmap = pd.DataFrame({'ts':
                             df.trip_start_timestamp.unique()})
        wmap['T/S'] = wmap.ts.apply(lambda x:
                                    w_df.loc[((w_df['StartTime(UTC)'] <= x) &
                                              (w_df['EndTime(UTC)'] >= x)),
                                             ['Type', 'Severity']].iloc[0, :]
                                        .to_numpy()
                                    if not w_df.loc[((w_df['StartTime(UTC)']
                                                      <= x) &
                                                     (w_df['EndTime(UTC)']
                                                      >= x)),
                                                    ['Type', 'Severity']]
                                               .empty
                                    else [None, None])
        wmap['Type'] = wmap['T/S'].apply(lambda x: x[0])
        wmap['Severity'] = wmap['T/S'].apply(lambda x: x[1])
        wmap.drop(columns=['T/S'], inplace=True)
        wmap.set_index('ts', inplace=True)

        # Merge Weather and TNP data
        df = pd.merge(df, wmap,
                      left_on='trip_start_timestamp',
                      right_index=True)

        # Write to CSV
        df.to_csv('tnp_sample.csv.gz', mode='a')
        print('COMPLETE:', d)

START: tnp_201901.db
COMPLETE: tnp_201901.db
START: tnp_201902.db
COMPLETE: tnp_201902.db
START: tnp_201903.db
COMPLETE: tnp_201903.db
START: tnp_201904.db
COMPLETE: tnp_201904.db
START: tnp_201905.db
COMPLETE: tnp_201905.db
START: tnp_201906.db
COMPLETE: tnp_201906.db
START: tnp_201907.db
COMPLETE: tnp_201907.db
START: tnp_201908.db
COMPLETE: tnp_201908.db
START: tnp_201909.db
COMPLETE: tnp_201909.db
START: tnp_201910.db
COMPLETE: tnp_201910.db
START: tnp_201911.db
COMPLETE: tnp_201911.db
START: tnp_201912.db
COMPLETE: tnp_201912.db


## Taxi Dataset

For the taxi dataset, we'll also randomly sample 2% from the whole database.

In [9]:
taxi_db = '/mnt/processed/private/msds2021/lt6/chicago-dataset/taxi/taxi.db'
with sqlite3.connect(taxi_db) as conn:
    print('START: taxi.db')
    sql = f"""SELECT * FROM taxi WHERE trip_id IN
                (SELECT trip_id FROM taxi ORDER BY RANDOM()
                 LIMIT (SELECT ROUND(COUNT(*) * 0.02) FROM taxi))"""
    df = pd.read_sql(sql, conn,
                     parse_dates=[x'trip_start_timestamp',
                                  'trip_end_timestamp'])

    # Build the Weather Mapper
    wmap = pd.DataFrame({'ts':
                         df.trip_start_timestamp.unique()})
    wmap['T/S'] = wmap.ts.apply(lambda x:
                                w_df.loc[((w_df['StartTime(UTC)'] <= x) &
                                          (w_df['EndTime(UTC)'] >= x)),
                                         ['Type', 'Severity']].iloc[0, :]
                                    .to_numpy()
                                if not w_df.loc[((w_df['StartTime(UTC)']
                                                  <= x) &
                                                 (w_df['EndTime(UTC)']
                                                  >= x)),
                                                ['Type', 'Severity']]
                                           .empty
                                else [None, None])
    wmap['Type'] = wmap['T/S'].apply(lambda x: x[0])
    wmap['Severity'] = wmap['T/S'].apply(lambda x: x[1])
    wmap.drop(columns=['T/S'], inplace=True)
    wmap.set_index('ts', inplace=True)

    # Merge Weather and TNP data
    df = pd.merge(df, wmap,
                  left_on='trip_start_timestamp',
                  right_index=True)

    # Write to CSV
    df.to_csv('taxi_sample.csv.gz', mode='a')
    print('COMPLETE: taxi.db')

START: taxi.db
COMPLETE: taxi.db
